In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import math
from sklearn.metrics import mean_squared_error

In [ ]:
dataset = pd.read_csv ('AAPL.csv')
dataset.head()

In [ ]:
training_set = dataset[:'2019'].iloc[:, 1:2].values
test_set = dataset[:'2020'].iloc[:, 1:2].values

In [ ]:
dataset["High"][:'2020'].plot(figsize = (16,4), legend = True)
dataset["High"][:'2021'].plot(figsize = (16,4), legend = True)
plt.legend(['Training set (Before 2020)', 'Test set (2020 and Beyond)'])
plt.title ('IBM stock price')
plt.show()

In [ ]:
sc = MinMaxScaler(feature_range= (0,1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
X_train = []
y_train = []
for i in range (60, 2769):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train, y_train = np.array(X_train), np.array(y_train)


In [ ]:
X_train = np.reshape(X_train, (X_train.reshape[0], X_train.shape[1],1))

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units=50, return_sequences=True, input_shape = (X_train.shape[1],1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=1))

regressor.compile(optimizer='rmsprop', loss='mean_squared_error')
regressor.fit(X_train, y_train, epochs= 50, batch_size= 32)

In [ ]:
dataset_total = pd.concat((dataset["High"][:'2016'], dataset["High"]['2017':]), axis = 0)
inputs = dataset_total[len(dataset_total)-len(test_set) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

In [ ]:
X_test = []

for i in range (60, 311):
    X_test.append(inputs[i-60:i,0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)